In [ ]:
import os, shutil, math, random, logging, torch
import numpy as np
import torch.nn as nn
import pandas as pd
import sklearn.preprocessing as prep
import matplotlib.pyplot as plt

From train 80%, validation 20%
<br>
Modified to train 60%, validation 20%, test 20% => Using "simple holdout validation"

Reference:
[Time series prediction](https://peaceful0907.medium.com/time-series-prediction-lstm%E7%9A%84%E5%90%84%E7%A8%AE%E7%94%A8%E6%B3%95-ed36f0370204)

![moving_window](Images/moving_window.png)

In [ ]:
def create_sequences(data, n_past, n_forecast, col_index):
    X, Y = [], []
    L = len(data)
    for i in range(L-(n_past+n_forecast)): # 1 day every step: i = 0, 1, 2, ..., L - (n_past+n_forecast)
        X.append(data[i:i+n_past]) # Input Sequence, using n_past days as input
        Y.append(data[i+n_past:i+n_past+n_forecast][:,col_index]) # Output Sequence, predicting n_forecast days as output

    return torch.Tensor(np.array(X)), torch.Tensor(np.array(Y))

In [ ]:
def preprocess(data_trend, train_ratio, test_ratio, n_past, n_forecast, col_index):
    scaler = prep.StandardScaler()
    data_trend = scaler.fit_transform(data_trend) # standardization

    train_index = int(len(data_trend)*train_ratio)
    test_index = int(train_index+len(data_trend)*test_ratio)

    train_data = data_trend[:train_index]
    test_data = data_trend[train_index:test_index]
    val_data = data_trend[test_index:]

    print(f'train_data is data_trend[:{train_index}], shape is {train_data.shape}')
    print(f'test_data is data_trend[{train_index}:{test_index}], shape is {test_data.shape}')
    print(f'val_data is data_trend[{test_index}:], shape is {val_data.shape}')

    X_train, Y_train = create_sequences(train_data, n_past, n_forecast, col_index)
    X_test, Y_test = create_sequences(test_data, n_past, n_forecast, col_index)
    X_val, Y_val = create_sequences(val_data, n_past, n_forecast, col_index)

    return X_train, Y_train, X_test, Y_test, X_val, Y_val

In [ ]:
df = pd.read_csv('2330.TW.csv')

data = df[[c for c in df.columns if c not in ['Date', 'Adj Close']]].values

# col_index = 3
# 0: Open, 1: High, 2: Low, 3: Close, 4: Volume
# 5 features to predict "Close"
X_train, Y_train, X_test, Y_test, X_val, Y_val = preprocess(data, train_ratio=0.6, test_ratio=0.2, n_past=20, n_forecast=5, col_index=3)

batch_size = 32

train_set = torch.utils.data.TensorDataset(X_train, Y_train)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False)
test_set = torch.utils.data.TensorDataset(X_test, Y_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)
val_set = torch.utils.data.TensorDataset(X_val, Y_val)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

print(len(train_loader), len(test_loader), len(val_loader))
3571/32, 1190/32, 1191/32

In [ ]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape, X_val.shape, Y_val.shape

In [ ]:
df.head() # before removing Date and Adj Close

In [ ]:
plt.plot(df['Close']) # plot the closing price of TSMC
plt.title('TSMC Stock Price')
plt.xlabel('Days passed')
plt.ylabel('Price')
plt.show()

Transformer-Decoder Architecture

Reference:
[Transformers for Time-series Forecasting](https://medium.com/mlearning-ai/transformer-implementation-for-time-series-forecasting-a9db2db5c820)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # input and output shape: [batch_size, seq_len, d_model]
        x = x.permute(1, 0, 2) # Change shape to [seq_len, batch_size, d_model]
        x = x + self.pe[:x.size(0)]
        x = x.permute(1, 0, 2) # Revert shape back to [batch_size, seq_len, d_model]
        return self.dropout(x)

In [ ]:
class Transformer(nn.Module):
    def __init__(self, d_model, nhead, dropout, num_layers):
        super(Transformer, self).__init__()

        self.encoder_layer = torch.nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True)
        self.transformer_encoder = torch.nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.decoder_layer = torch.nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True)
        self.transformer_decoder = torch.nn.TransformerDecoder(self.decoder_layer, num_layers=num_layers)
        self.decoder = torch.nn.Linear(d_model, 1) # ask the decoder output 1 or 5 forecast days

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.decoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()

    def forward(self, src, tgt): # ask whether forward method is correct
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(len(src))
        memory = self.transformer_encoder(src) # memory is the output of encoder
        output = self.transformer_decoder(tgt, memory, tgt_mask) # the decoder takes in the memory, tgt, and tgt_mask
        output = self.decoder(output) # forecast
        return output

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.expansion = torch.nn.Linear(in_features=5, out_features=512)
        self.positional_encoding = PositionalEncoding(d_model=512, dropout=0.1, max_len=5000)
        self.transformer = Transformer(d_model=512, nhead=8, dropout=0.1, num_layers=6)

    def forward(self, src, tgt): # how to fit data to model
        src = self.expansion(src)
        tgt = self.expansion(tgt)
        print(src.shape, tgt.shape)
        pe = self.positional_encoding(src)
        output = self.transformer(pe, tgt)
        return output

model = Model()
model

In [ ]:
X_train[:32].shape, Y_train[:32].shape

In [ ]:
output = model(X_train[:32], Y_train[:32])
output.shape

In [ ]:
# train on cuda

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU or CPU
print('Using device:', device)

with torch.cuda.device(device):
    torch.cuda.empty_cache()

In [ ]:
'stop here'

helpers.py

In [ ]:
def log_loss(loss_val : float, path_to_save_loss : str, train : bool = True):
    if train:
        file_name = 'train_loss.txt'
    else:
        file_name = 'val_loss.txt'

    path_to_file = path_to_save_loss+file_name
    os.makedirs(os.path.dirname(path_to_file), exist_ok=True)
    with open(path_to_file, 'a') as f:
        f.write(str(loss_val)+'\n')
        f.close()

def EMA(values, alpha=0.1):
    ema_values = [values[0]]
    for idx, item in enumerate(values[1:]):
        ema_values.append(alpha*item + (1-alpha)*ema_values[idx])
    return ema_values

# Remove all files from previous executions and re-run the model.
def clean_directory():

    if os.path.exists('save_loss'):
        shutil.rmtree('save_loss')
    if os.path.exists('save_model'):
        shutil.rmtree('save_model')
    if os.path.exists('save_predictions'):
        shutil.rmtree('save_predictions')
    os.mkdir('save_loss')
    os.mkdir('save_model')
    os.mkdir('save_predictions')

plot.py

In [ ]:
def plot_loss(path_to_save, train=True):
    plt.rcParams.update({'font.size': 10})
    with open(path_to_save + '/train_loss.txt', 'r') as f:
        loss_list = [float(line) for line in f.readlines()]
    if train:
        title = 'Train'
    else:
        title = 'Validation'
    EMA_loss = EMA(loss_list)
    plt.plot(loss_list, label = 'loss')
    plt.plot(EMA_loss, label='EMA loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(title+'_loss')
    plt.savefig(path_to_save+f'/{title}.png')
    plt.close()

def plot_prediction(title, path_to_save, src, tgt, prediction, sensor_number, index_in, index_tar):

    idx_scr = index_in[0, 1:].tolist()
    idx_tgt = index_tar[0].tolist()
    idx_pred = [i for i in range(idx_scr[0] +1, idx_tgt[-1])] #t2 - t61

    plt.figure(figsize=(15,6))
    plt.rcParams.update({'font.size' : 16})

    # connect with last elemenet in src
    # tgt = np.append(src[-1], tgt.flatten())
    # prediction = np.append(src[-1], prediction.flatten())

    # plotting
    plt.plot(idx_scr, src, '-', color = 'blue', label = 'Input', linewidth=2)
    plt.plot(idx_tgt, tgt, '-', color = 'indigo', label = 'Target', linewidth=2)
    plt.plot(idx_pred, prediction,'--', color = 'limegreen', label = 'Forecast', linewidth=2)

    #formatting
    plt.grid(b=True, which='major', linestyle = 'solid')
    plt.minorticks_on()
    plt.grid(b=True, which='minor', linestyle = 'dashed', alpha=0.5)
    plt.xlabel('Time Elapsed')
    plt.ylabel('Humidity (%)')
    plt.legend()
    plt.title('Forecast from Sensor ' + str(sensor_number[0]))

    # save
    plt.savefig(path_to_save+f'Prediction_{title}.png')
    plt.close()

def plot_training(epoch, path_to_save, src, prediction, sensor_number, index_in, index_tar):

    # idx_scr = index_in.tolist()[0]
    # idx_tar = index_tar.tolist()[0]
    # idx_pred = idx_scr.append(idx_tar.append([idx_tar[-1] + 1]))

    idx_scr = [i for i in range(len(src))]
    idx_pred = [i for i in range(1, len(prediction)+1)]

    plt.figure(figsize=(15,6))
    plt.rcParams.update({'font.size' : 18})
    plt.grid(b=True, which='major', linestyle = '-')
    plt.grid(b=True, which='minor', linestyle = '--', alpha=0.5)
    plt.minorticks_on()

    plt.plot(idx_scr, src, 'o-.', color = 'blue', label = 'input sequence', linewidth=1)
    plt.plot(idx_pred, prediction, 'o-.', color = 'limegreen', label = 'prediction sequence', linewidth=1)

    plt.title('Teaching Forcing from Sensor ' + str(sensor_number[0]) + ', Epoch ' + str(epoch))
    plt.xlabel('Time Elapsed')
    plt.ylabel('Humidity (%)')
    plt.legend()
    plt.savefig(path_to_save+f'/Epoch_{str(epoch)}.png')
    plt.close()

def plot_training_3(epoch, path_to_save, src, sampled_src, prediction, sensor_number, index_in, index_tar):

    # idx_scr = index_in.tolist()[0]
    # idx_tar = index_tar.tolist()[0]
    # idx_pred = idx_scr.append(idx_tar.append([idx_tar[-1] + 1]))

    idx_scr = [i for i in range(len(src))]
    idx_pred = [i for i in range(1, len(prediction)+1)]
    idx_sampled_src = [i for i in range(len(sampled_src))]

    plt.figure(figsize=(15,6))
    plt.rcParams.update({'font.size' : 18})
    plt.grid(b=True, which='major', linestyle = '-')
    plt.grid(b=True, which='minor', linestyle = '--', alpha=0.5)
    plt.minorticks_on()

    ## REMOVE DROPOUT FOR THIS PLOT TO APPEAR AS EXPECTED !! DROPOUT INTERFERES WITH HOW THE SAMPLED SOURCES ARE PLOTTED
    plt.plot(idx_sampled_src, sampled_src, 'o-.', color='red', label = 'sampled source', linewidth=1, markersize=10)
    plt.plot(idx_scr, src, 'o-.', color = 'blue', label = 'input sequence', linewidth=1)
    plt.plot(idx_pred, prediction, 'o-.', color = 'limegreen', label = 'prediction sequence', linewidth=1)
    plt.title('Teaching Forcing from Sensor ' + str(sensor_number[0]) + ', Epoch ' + str(epoch))
    plt.xlabel('Time Elapsed')
    plt.ylabel('Humidity (%)')
    plt.legend()
    plt.savefig(path_to_save+f'/Epoch_{str(epoch)}.png')
    plt.close()

model.py

In [ ]:
class Transformer(nn.Module):
    # d_model : number of features
    def __init__(self,feature_size=256,num_layers=3,dropout=0.1):
        super(Transformer, self).__init__()

        self.encoder_layer = torch.nn.TransformerEncoderLayer(d_model=feature_size, nhead=8, dropout=dropout, batch_first=True)
        self.transformer_encoder = torch.nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.decoder = torch.nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, device):

        mask = self._generate_square_subsequent_mask(len(src)).to(device)
        output = self.transformer_encoder(src,mask)
        output = self.decoder(output)
        return output

train_with_sampling.py

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(name)s %(message)s', datefmt='[%Y-%m-%d %H:%M:%S]')
logger = logging.getLogger(__name__)

def flip_from_probability(p):
    return True if random.random() < p else False

def transformer(dataloader, EPOCH, k, frequency, path_to_save_model, path_to_save_loss, path_to_save_predictions, device):

    device = torch.device(device)

    model = Transformer().double().to(device)
    optimizer = torch.optim.Adam(model.parameters())
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=200)
    criterion = torch.nn.MSELoss()
    best_model = ''
    min_train_loss = float('inf')

    for epoch in range(EPOCH + 1):
        train_loss = 0
        val_loss = 0

        ## TRAIN -- TEACHER FORCING
        model.train()
        for index_in, index_tar, _input, target, sensor_number in dataloader:

            # Shape of _input : [batch, input_length, feature]
            # Desired input for model: [input_length, batch, feature]

            optimizer.zero_grad()
            src = _input.permute(1,0,2).double().to(device)[:-1,:,:] # torch.Size([24, 1, 7])
            target = _input.permute(1,0,2).double().to(device)[1:,:,:] # src shifted by 1.
            sampled_src = src[:1, :, :] #t0 torch.Size([1, 1, 7])

            for i in range(len(target)-1):

                prediction = model(sampled_src, device) # torch.Size([1xw, 1, 1])
                # for p1, p2 in zip(params, model.parameters()):
                #     if p1.data.ne(p2.data).sum() > 0:
                #         ic(False)
                # ic(True)
                # ic(i, sampled_src[:,:,0], prediction)
                # time.sleep(1)
                '''
                # to update model at every step
                # loss = criterion(prediction, target[:i+1,:,:1])
                # loss.backward()
                # optimizer.step()
                '''

                if i < 24: # One day, enough data to make inferences about cycles
                    prob_true_val = True
                else:
                    ## coin flip
                    v = k/(k+math.exp(epoch/k)) # probability of heads/tails depends on the epoch, evolves with time.
                    prob_true_val = flip_from_probability(v) # starts with over 95 % probability of true val for each flip in epoch 0.
                    ## if using true value as new value

                if prob_true_val: # Using true value as next value
                    sampled_src = torch.cat((sampled_src.detach(), src[i+1, :, :].unsqueeze(0).detach()))
                else: ## using prediction as new value
                    positional_encodings_new_val = src[i+1,:,1:].unsqueeze(0)
                    predicted_humidity = torch.cat((prediction[-1,:,:].unsqueeze(0), positional_encodings_new_val), dim=2)
                    sampled_src = torch.cat((sampled_src.detach(), predicted_humidity.detach()))

            'To update model after each sequence'
            loss = criterion(target[:-1,:,0].unsqueeze(-1), prediction)
            loss.backward()
            optimizer.step()
            train_loss += loss.detach().item()

        if train_loss < min_train_loss:
            torch.save(model.state_dict(), path_to_save_model + f'best_train_{epoch}.pth')
            torch.save(optimizer.state_dict(), path_to_save_model + f'optimizer_{epoch}.pth')
            min_train_loss = train_loss
            best_model = f'best_train_{epoch}.pth'


        if epoch % 10 == 0: # Plot 1-Step Predictions

            logger.info(f'Epoch: {epoch}, Training loss: {train_loss}')
            scaler = load('scalar_item.joblib')
            sampled_src_humidity = scaler.inverse_transform(sampled_src[:,:,0].cpu()) #torch.Size([35, 1, 7])
            src_humidity = scaler.inverse_transform(src[:,:,0].cpu()) #torch.Size([35, 1, 7])
            target_humidity = scaler.inverse_transform(target[:,:,0].cpu()) #torch.Size([35, 1, 7])
            prediction_humidity = scaler.inverse_transform(prediction[:,:,0].detach().cpu().numpy()) #torch.Size([35, 1, 7])
            plot_training_3(epoch, path_to_save_predictions, src_humidity, sampled_src_humidity, prediction_humidity, sensor_number, index_in, index_tar)

        train_loss /= len(dataloader)
        log_loss(train_loss, path_to_save_loss, train=True)

    plot_loss(path_to_save_loss, train=True)
    return best_model

train_teacher_forcing.py

In [ ]:
# logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(name)s %(message)s', datefmt='[%Y-%m-%d %H:%M:%S]')
# logger = logging.getLogger(__name__)

def transformer(dataloader, EPOCH, k, frequency, path_to_save_model, path_to_save_loss, path_to_save_predictions, device):

    device = torch.device(device)

    model = Transformer().double().to(device)
    optimizer = torch.optim.AdamW(model.parameters())
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=200)
    criterion = torch.nn.MSELoss()
    best_model = ''
    min_train_loss = float('inf')

    for epoch in range(EPOCH + 1):
        train_loss = 0
        val_loss = 0

        ## TRAIN -- TEACHER FORCING
        model.train()
        for index_in, index_tar, _input, target, sensor_number in dataloader:

            # Shape of _input : [batch, input_length, feature]
            # Desired input for model: [input_length, batch, feature]

            optimizer.zero_grad()
            src = _input.permute(1,0,2).double().to(device)[:-1,:,:] # torch.Size([24, 1, 7])
            target = _input.permute(1,0,2).double().to(device)[1:,:,:] # src shifted by 1.
            sampled_src = src[:1, :, :] #t0 torch.Size([1, 1, 7])

            for i in range(len(target)-1):

                prediction = model(sampled_src, device) # torch.Size([1xw, 1, 1])
                # for p1, p2 in zip(params, model.parameters()):
                #     if p1.data.ne(p2.data).sum() > 0:
                #         ic(False)
                # ic(True)
                # ic(i, sampled_src[:,:,0], prediction)
                # time.sleep(1)
                '''
                # to update model at every step
                # loss = criterion(prediction, target[:i+1,:,:1])
                # loss.backward()
                # optimizer.step()
                '''

                if i < 24: # One day, enough data to make inferences about cycles
                    prob_true_val = True
                else:
                    ## coin flip
                    v = k/(k+math.exp(epoch/k)) # probability of heads/tails depends on the epoch, evolves with time.
                    prob_true_val = flip_from_probability(v) # starts with over 95 % probability of true val for each flip in epoch 0.
                    ## if using true value as new value

                if prob_true_val: # Using true value as next value
                    sampled_src = torch.cat((sampled_src.detach(), src[i+1, :, :].unsqueeze(0).detach()))
                else: ## using prediction as new value
                    positional_encodings_new_val = src[i+1,:,1:].unsqueeze(0)
                    predicted_humidity = torch.cat((prediction[-1,:,:].unsqueeze(0), positional_encodings_new_val), dim=2)
                    sampled_src = torch.cat((sampled_src.detach(), predicted_humidity.detach()))

            'To update model after each sequence'
            loss = criterion(target[:-1,:,0].unsqueeze(-1), prediction)
            loss.backward()
            optimizer.step()
            train_loss += loss.detach().item()

        if train_loss < min_train_loss:
            torch.save(model.state_dict(), path_to_save_model + f'best_train_{epoch}.pth')
            torch.save(optimizer.state_dict(), path_to_save_model + f'optimizer_{epoch}.pth')
            min_train_loss = train_loss
            best_model = f'best_train_{epoch}.pth'


        # if epoch % 10 == 0: # Plot 1-Step Predictions

        #     logger.info(f'Epoch: {epoch}, Training loss: {train_loss}')
        #     scaler = load('scalar_item.joblib')
        #     sampled_src_humidity = scaler.inverse_transform(sampled_src[:,:,0].cpu()) #torch.Size([35, 1, 7])
        #     src_humidity = scaler.inverse_transform(src[:,:,0].cpu()) #torch.Size([35, 1, 7])
        #     target_humidity = scaler.inverse_transform(target[:,:,0].cpu()) #torch.Size([35, 1, 7])
        #     prediction_humidity = scaler.inverse_transform(prediction[:,:,0].detach().cpu().numpy()) #torch.Size([35, 1, 7])
        #     plot_training_3(epoch, path_to_save_predictions, src_humidity, sampled_src_humidity, prediction_humidity, sensor_number, index_in, index_tar)

        train_loss /= len(dataloader)
    #     log_loss(train_loss, path_to_save_loss, train=True)

    # plot_loss(path_to_save_loss, train=True)
    return best_model

inference.py

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(name)s %(message)s', datefmt='[%Y-%m-%d %H:%M:%S]')
logger = logging.getLogger(__name__)

def inference(path_to_save_predictions, forecast_window, dataloader, device, path_to_save_model, best_model):

    device = torch.device(device)

    model = Transformer().double().to(device)
    model.load_state_dict(torch.load(path_to_save_model+best_model))
    criterion = torch.nn.MSELoss()

    val_loss = 0
    with torch.no_grad():

        model.eval()
        # for plot in range(25):

            for index_in, index_tar, _input, target, sensor_number in dataloader:

                # starting from 1 so that src matches with target, but has same length as when training
                src = _input.permute(1,0,2).double().to(device)[1:, :, :] # 47, 1, 7: t1 -- t47
                target = target.permute(1,0,2).double().to(device) # t48 - t59

                next_input_model = src
                all_predictions = []

                for i in range(forecast_window - 1):

                    prediction = model(next_input_model, device) # 47,1,1: t2' - t48'

                    if all_predictions == []:
                        all_predictions = prediction # 47,1,1: t2' - t48'
                    else:
                        all_predictions = torch.cat((all_predictions, prediction[-1,:,:].unsqueeze(0))) # 47+,1,1: t2' - t48', t49', t50'

                    pos_encoding_old_vals = src[i+1:, :, 1:] # 46, 1, 6, pop positional encoding first value: t2 -- t47
                    pos_encoding_new_val = target[i + 1, :, 1:].unsqueeze(1) # 1, 1, 6, append positional encoding of last predicted value: t48
                    pos_encodings = torch.cat((pos_encoding_old_vals, pos_encoding_new_val)) # 47, 1, 6 positional encodings matched with prediction: t2 -- t48

                    next_input_model = torch.cat((src[i+1:, :, 0].unsqueeze(-1), prediction[-1,:,:].unsqueeze(0))) #t2 -- t47, t48'
                    next_input_model = torch.cat((next_input_model, pos_encodings), dim = 2) # 47, 1, 7 input for next round

                true = torch.cat((src[1:,:,0],target[:-1,:,0]))
                loss = criterion(true, all_predictions[:,:,0])
                val_loss += loss

            val_loss = val_loss/10
            scaler = load('scalar_item.joblib')
            src_humidity = scaler.inverse_transform(src[:,:,0].cpu())
            target_humidity = scaler.inverse_transform(target[:,:,0].cpu())
            prediction_humidity = scaler.inverse_transform(all_predictions[:,:,0].detach().cpu().numpy())
            plot_prediction(plot, path_to_save_predictions, src_humidity, target_humidity, prediction_humidity, sensor_number, index_in, index_tar)

        logger.info(f'Loss On Unseen Dataset: {val_loss.item()}')